In [4]:
import pandas as pd
import re
import ast

# ==========================================
# 1. Load the Datasets
# ==========================================

# Load CSV (Orders)
orders_df = pd.read_csv('orders.csv')

# Load JSON (Users)
users_df = pd.read_json('users.json')

# Load SQL (Restaurants) - Parsing the INSERT statements
restaurant_data = []
with open('restaurants.sql', 'r') as f:
    for line in f:
        if line.strip().startswith("INSERT INTO restaurants VALUES"):
            # Extract content inside parentheses: (id, 'Name', 'Cuisine', rating)
            match = re.search(r"VALUES\s*\((.*)\);", line, re.IGNORECASE)
            if match:
                values_str = match.group(1)
                try:
                    # Safely evaluate the string as a tuple
                    values = ast.literal_eval(f"({values_str})")
                    restaurant_data.append(values)
                except Exception as e:
                    print(f"Error parsing line: {line} -> {e}")

# Create DataFrame from parsed SQL data
restaurants_df = pd.DataFrame(restaurant_data, columns=['restaurant_id', 'restaurant_name', 'cuisine', 'rating'])

# ==========================================
# 2. Merge the Data
# ==========================================

# Merge Orders with Users (Left Join)
df = pd.merge(orders_df, users_df, on='user_id', how='left')

# Merge the result with Restaurants (Left Join)
# Note: Orders has 'restaurant_id', Restaurants has 'restaurant_id'
df = pd.merge(df, restaurants_df, on='restaurant_id', how='left', suffixes=('_order', '_rest'))

# Save the final dataset
df.to_csv('final_food_delivery_dataset.csv', index=False)
print("Dataset created and saved as 'final_food_delivery_dataset.csv'")
print(f"Total Rows: {len(df)}")

# ==========================================
# 3. Data Analysis & Answers
# ==========================================

print("\n--- Numerical Analysis ---")

# 1. Total orders by Gold members
gold_orders = df[df['membership'] == 'Gold'].shape[0]
print(f"Total Gold Member Orders: {gold_orders}")

# 2. Total revenue from Hyderabad
hyd_revenue = df[df['city'] == 'Hyderabad']['total_amount'].sum()
print(f"Total Revenue (Hyderabad): {round(hyd_revenue)}")

# 3. Distinct users who placed orders
distinct_users = df['user_id'].nunique()
print(f"Distinct Users: {distinct_users}")

# 4. Average order value for Gold members
avg_gold_val = df[df['membership'] == 'Gold']['total_amount'].mean()
print(f"Avg Order Value (Gold): {round(avg_gold_val, 2)}")

# 5. Orders for restaurants with rating >= 4.5
high_rated_orders = df[df['rating'] >= 4.5].shape[0]
print(f"Orders for Rating >= 4.5: {high_rated_orders}")

# 6. Orders in top revenue city (Gold members only)
gold_df = df[df['membership'] == 'Gold']
top_city_gold = gold_df.groupby('city')['total_amount'].sum().idxmax()
top_city_orders = gold_df[gold_df['city'] == top_city_gold].shape[0]
print(f"Orders in Top Gold City ({top_city_gold}): {top_city_orders}")

print("\n--- Additional Insights (for MCQs) ---")

# City with highest Gold revenue
print("Top City by Gold Revenue:", gold_df.groupby('city')['total_amount'].sum().idxmax())

# Cuisine with highest avg order value
print("Cuisine with Highest Avg Order Value:", df.groupby('cuisine')['total_amount'].mean().idxmax())

# Users with > 1000 total spend
user_totals = df.groupby('user_id')['total_amount'].sum()
print("Users with > 1000 spend:", (user_totals > 1000).sum())

# Revenue by rating range
# (You can create bins here if needed, but simple inspection works for the question)
print("Revenue by Rating (approx):")
print(df.groupby(pd.cut(df['rating'], bins=[2.9, 3.5, 4.0, 4.5, 5.0]))['total_amount'].sum())

Dataset created and saved as 'final_food_delivery_dataset.csv'
Total Rows: 10000

--- Numerical Analysis ---
Total Gold Member Orders: 4987
Total Revenue (Hyderabad): 1889367
Distinct Users: 2883
Avg Order Value (Gold): 797.15
Orders for Rating >= 4.5: 3374
Orders in Top Gold City (Chennai): 1337

--- Additional Insights (for MCQs) ---
Top City by Gold Revenue: Chennai
Cuisine with Highest Avg Order Value: Mexican
Users with > 1000 spend: 2544
Revenue by Rating (approx):
rating
(2.9, 3.5]    2136772.70
(3.5, 4.0]    1717494.41
(4.0, 4.5]    1960326.26
(4.5, 5.0]    2197030.75
Name: total_amount, dtype: float64


/tmp/ipython-input-3589731330.py:97: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby(pd.cut(df['rating'], bins=[2.9, 3.5, 4.0, 4.5, 5.0]))['total_amount'].sum())
